# Fine-tune Bart on XSUM small dataset

## Libraries and environment preparation

In [1]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [2]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Thu Mar 10 16:56:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "facebook/bart-base"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [6]:
# import dataset
raw_datasets = datasets.load_dataset("xsum")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

## Preprocessing the data

In [8]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [9]:
# tokenlize inputs into map

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_train = raw_datasets["train"].select(range(10000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["id", "document", "summary"])

  0%|          | 0/10 [00:00<?, ?ba/s]

In [12]:
tokenized_val = raw_datasets["validation"].select(range(1000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [13]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [14]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
# keep track with wandb
wandb.init(project="BART-sum")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


Define `Seq2SeqTrainer` to compute the metrics from the predictions, and also do a bit of pre-processing to decode the predictions into texts:

In [16]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-cnn",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=2000, 
    save_steps=4000,
    eval_steps=4000,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [17]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [18]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 16
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 40000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
4000,2.095800,2.185836,33.739300,12.709000,27.575900,27.557700,19.396000
8000,1.553500,2.348021,33.674400,12.526300,27.434200,27.459600,19.626000
12000,1.072900,2.449652,32.943600,12.400800,26.972700,26.982500,19.677000
16000,0.783500,2.659142,33.357000,12.296100,26.873700,26.898500,19.817000
20000,0.564400,2.717460,32.868800,11.914000,26.712600,26.719100,19.802000
24000,0.388300,2.865485,33.392800,12.160600,27.024900,27.055100,19.769000
28000,0.284300,2.979181,33.265300,11.969700,26.937700,26.925600,19.753000
32000,0.200200,3.035394,33.312500,12.303700,27.124600,27.124400,19.836000
36000,0.158000,3.084723,33.254200,12.328200,27.037200,27.010200,19.747000
40000,0.129400,3.103288,33.631500,12.550200,27.452700,27.447600,19.793000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-cnn/checkpoint-4000
Configuration saved in bart-base-finetuned-cnn/checkpoint-4000/config.json
Model weights saved in bart-base-finetuned-cnn/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-cnn/checkpoint-4000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-cnn/checkpoint-4000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-cnn/checkpoint-8000
Configuration saved in bart-base-finetuned-cnn/checkpoint-8000/config.json
Model weights saved in bart-base-finetuned-cnn/checkpoint-8000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-cnn/checkpoint-8000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-cnn/checkpoint-8000/special_tokens_map.json
***** Running Evaluation *****
  Num examp

TrainOutput(global_step=40000, training_loss=0.7913939949035644, metrics={'train_runtime': 13090.6079, 'train_samples_per_second': 12.223, 'train_steps_per_second': 3.056, 'total_flos': 7.360706607980544e+16, 'train_loss': 0.7913939949035644, 'epoch': 16.0})

In [ ]:
wandb.finish()

In [ ]:
!ls -lh

total 12K
drwxr-xr-x 5 root root 4.0K Feb 17 05:51 bart-base-finetuned-cnn
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data
drwxr-xr-x 3 root root 4.0K Feb 16 19:04 wandb


In [ ]:
!zip -r /content/bart-base-finetuned-cnn.zip /content/bart-base-finetuned-cnn/checkpoint-70000/

  adding: content/bart-base-finetuned-cnn/checkpoint-70000/ (stored 0%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/config.json (deflated 63%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/training_args.bin (deflated 49%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/trainer_state.json (deflated 80%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/scaler.pt (deflated 55%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/tokenizer_config.json (deflated 45%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/special_tokens_map.json (deflated 50%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/tokenizer.json (deflated 72%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/merges.txt (deflated 53%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/scheduler.pt (deflated 49%)
  adding: content/bart-base-finetuned-cnn/checkpoint-70000/rng_state.pth (deflated 28%)
  adding: content/bart-base-fi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp bart-base-finetuned-cnn.zip '/content/drive/My Drive/weights/'